In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import beta, norm, ks_2samp, wasserstein_distance
import matplotlib.pyplot as plt
import os

os.makedirs("plots", exist_ok=True)

def generate_theta_y_n_large(batch_size=256, n_min=50, n_max=100_000, log_uniform=True):
    if log_uniform:
        log_n = np.random.uniform(np.log(n_min), np.log(n_max), size=(batch_size, 1))
        n = np.round(np.exp(log_n)).astype(np.float32)
    else:
        n = np.random.randint(n_min, n_max + 1, size=(batch_size, 1)).astype(np.float32)
    theta = np.random.uniform(0, 1, size=(batch_size, 1)).astype(np.float32)
    y = np.random.binomial(n=n.astype(int), p=theta).astype(np.float32)
    return y, n, theta

def gmm_log_likelihood(theta, logits, mus, log_sigmas):
    log_sigmas = torch.clamp(log_sigmas, min=-5, max=3)
    sigmas = torch.exp(log_sigmas)
    pi = torch.softmax(logits, dim=1)
    th = theta.unsqueeze(1).expand(-1, mus.size(1))
    logp = -0.5 * ((th - mus) / sigmas) ** 2 - log_sigmas - 0.5 * np.log(2 * np.pi)
    wlogp = logp + torch.log(pi + 1e-10)
    return -torch.mean(torch.logsumexp(wlogp, dim=1))

class GMMInferenceNet(nn.Module):
    def __init__(self, num_components=3, hidden_dims=(512, 512, 256, 128)):
        super().__init__()
        self.K = num_components
        layers = []
        d = 2
        for h in hidden_dims:
            layers += [nn.Linear(d, h), nn.ReLU()]
            d = h
        layers.append(nn.Linear(d, 3 * self.K))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        out = self.net(x)
        return out[:, :self.K], out[:, self.K:2*self.K], out[:, 2*self.K:]

def train_gmm_network(model, optimizer, num_epochs=200, batch_size=256, n_min=50, n_max=100_000, steps_per_epoch=100):
    for epoch in range(num_epochs):
        total_loss = 0.0
        for _ in range(steps_per_epoch):
            y_np, n_np, theta_np = generate_theta_y_n_large(batch_size, n_min, n_max)
            x = torch.cat([
                torch.from_numpy(y_np / n_np),
                torch.from_numpy(np.log(n_np))
            ], dim=1).float()
            theta = torch.from_numpy(theta_np).squeeze(1).float()
            logits, mus, logs = model(x)
            pi = torch.softmax(logits, dim=1)
            nll = gmm_log_likelihood(theta, logits, mus, logs)
            entropy = -torch.sum(pi * torch.log(pi + 1e-8), dim=1).mean()
            loss = nll - 0.01 * entropy
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"[GMM] Epoch {epoch + 1}/{num_epochs} | Loss: {total_loss / steps_per_epoch:.4f}")

def evaluate_gmm_fixed_n(model, fixed_n, y_values, theta_grid, num_components=3):
    plt.figure(figsize=(10, 3.5 * len(y_values)))
    for i, y in enumerate(y_values):
        alpha, beta_param = y + 1, fixed_n - y + 1
        true_pdf = beta.pdf(theta_grid, alpha, beta_param)
        x = torch.tensor([[y / fixed_n, np.log(fixed_n)]], dtype=torch.float32)
        with torch.no_grad():
            logits, mus, logs = model(x)
        pi = torch.softmax(logits, dim=1).numpy().flatten()
        mus = mus.numpy().flatten()
        sig = np.exp(logs.numpy()).flatten()
        gmm_pdf = sum(pi[k] * norm.pdf(theta_grid, mus[k], sig[k]) for k in range(num_components))
        ts = beta.rvs(alpha, beta_param, size=2000)
        ss = np.clip(np.hstack([
            np.random.normal(mus[k], sig[k], int(round(pi[k] * 2000)))
            for k in range(num_components)
        ]), 0, 1)
        ks, p = ks_2samp(ts, ss)
        wd = wasserstein_distance(ts, ss)
        ax = plt.subplot(len(y_values), 1, i + 1)
        ax.plot(theta_grid, true_pdf, 'b--')
        ax.plot(theta_grid, gmm_pdf, 'r-')
        ax.set_title(f"GMM Fixed n={fixed_n}, y={y} | KS={ks:.3f}, p={p:.3f}, W={wd:.3f}")
    plt.tight_layout()
    plt.savefig(f"plots/gmm_fixedN_{fixed_n}.png", dpi=300)
    plt.show()

def evaluate_gmm_unseen_n(model, test_cases, theta_grid, num_components=3):
    plt.figure(figsize=(10, 3.5 * len(test_cases)))
    for i, (n, y) in enumerate(test_cases):
        alpha, beta_param = y + 1, n - y + 1
        true_pdf = beta.pdf(theta_grid, alpha, beta_param)
        x = torch.tensor([[y / n, np.log(n)]], dtype=torch.float32)
        with torch.no_grad():
            logits, mus, logs = model(x)
        pi = torch.softmax(logits, dim=1).numpy().flatten()
        mus = mus.numpy().flatten()
        sig = np.exp(logs.numpy()).flatten()
        gmm_pdf = sum(pi[k] * norm.pdf(theta_grid, mus[k], sig[k]) for k in range(num_components))
        ts = beta.rvs(alpha, beta_param, size=2000)
        ss = np.clip(np.hstack([
            np.random.normal(mus[k], sig[k], int(round(pi[k] * 2000)))
            for k in range(num_components)
        ]), 0, 1)
        ks, p = ks_2samp(ts, ss)
        wd = wasserstein_distance(ts, ss)
        ax = plt.subplot(len(test_cases), 1, i + 1)
        ax.plot(theta_grid, true_pdf, 'b--')
        ax.plot(theta_grid, gmm_pdf, 'r-')
        ax.set_title(f"GMM Unseen n={n}, y={y} | KS={ks:.3f}, p={p:.3f}, W={wd:.3f}")
    plt.tight_layout()
    plt.savefig("plots/gmm_unseenN.png", dpi=300)
    plt.show()

K = 10  
hidden_dims = (512, 512, 256, 128)
gmm_model = GMMInferenceNet(num_components=K, hidden_dims=hidden_dims)
gmm_optimizer = optim.Adam(gmm_model.parameters(), lr=1e-3)
print("----- Training GMM Network -----")
train_gmm_network(gmm_model, gmm_optimizer)

theta_grid = np.linspace(0, 1, 500)
y_values_fixed = [1, 100, 2000, 8000]
test_cases_unseen = [(20000, 1), (50000, 25000), (200000, 150000)]

print("----- Evaluating GMM Fixed n=10000 -----")
evaluate_gmm_fixed_n(gmm_model, fixed_n=10000, y_values=y_values_fixed, theta_grid=theta_grid, num_components=K)
print("----- Evaluating GMM Unseen n's -----")
evaluate_gmm_unseen_n(gmm_model, test_cases_unseen, theta_grid, num_components=K)
